In [5]:
import models.vae_mnist as vae_mnist
from torchvision import transforms, datasets
import torch

In [6]:
transform = transforms.ToTensor()

# download data for training
mnist_data = datasets.MNIST(root='./todata', train=True,
                            download=True, transform = transform)

# set dataloader
data_loader = torch.utils.data.DataLoader(dataset = mnist_data,
                                          batch_size = 64,
                                          shuffle = True)
# download data for test
test_data = datasets.MNIST(root='./data', train=False,
                            download=True, transform = transform)

In [7]:
device = 'cpu'

latent_dim = 6

mnist_data = datasets.MNIST(root='./data', train=True,
                            download=True, transform = transform)

mnist_loader = torch.utils.data.DataLoader(dataset = mnist_data,
                                          batch_size = 64,
                                          shuffle = True)
test_data = datasets.MNIST(root='./data', train=False,
                            download=True, transform = transform)

conv_vae = vae_mnist.ConvVAE(latent_dim).to(device)
conv_vae.load_state_dict(torch.load('./pth/conv_vae_mnist.pth', map_location=device))

<All keys matched successfully>

In [8]:
print(conv_vae.encoder.modules)
print(conv_vae.decoder.modules)

<bound method Module.modules of Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (3): ReLU()
  (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (5): ReLU()
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=6272, out_features=1024, bias=True)
  (8): ReLU()
  (9): Linear(in_features=1024, out_features=12, bias=True)
)>
<bound method Module.modules of Sequential(
  (0): Linear(in_features=6, out_features=1024, bias=True)
  (1): ReLU()
  (2): Linear(in_features=1024, out_features=6272, bias=True)
  (3): ReLU()
  (4): Unflatten(dim=1, unflattened_size=(128, 7, 7))
  (5): ConvTranspose2d(128, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
  (6): ReLU()
  (7): ConvTranspose2d(64, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
  (8): ReLU()
  (9): ConvTranspose2d(32, 1, 